# 路径跟踪
在“避障”示例中我们已经学习如何进行数据收集，模型训练和的操作了，在这个示例中我们把数据分为了两类（“有障碍”和“无障碍”），我们把图片通过避障模型预测了这个图片是否有障碍，我们把这种方法叫做分类。

在这个示例里，我们还是会通过神经网络来进行训练! 不过，这次我们将学习另一种方法，叫做“regression”（回归），我们将通过回归的方法来让机器人沿着一条路前进。
> 什么叫做回归？
在分类问题中我们会把神经网络的输出层改为我们想预测的类别数量，并且在训练时我们会给训练图片的标签用0和1的方式来表达，比如“避障”示例中，“有障碍”的标签为[1,0],“无障碍”的标签为[0,1],但是我们在学习中并没有看到这样的操作，因为我们所导入的模块已经自动帮我们把类别标记为以0和1这种方式了。相比分类，回归会更好理解，比如现在我们有120张2种类型的卡片，一种类型卡片上写着“1”，另一种类型卡片写着“2”，但是图片的颜色或者数字的形状不相同，现在我们要随机拿其中100张图片进行训练后得到的模型来预测剩下的20张卡片分别是什么数字，在这种情况下我们使用回归的方法就会更方便，直接把卡片上有“1”的图片标记为1，有“2”的图片标记为2，然后再通过学习训练后就会得到一个回归的模型，然后再通过模型进行预测就会预测出这张卡片的数字是多少了。

路径跟踪的具体步骤：
1. 打开机器人的摄像头；
>  从“避障”示例中可以知道，数据的多样性是很重要的!
2. 将机器人放置在路径的不同位置，不同角度等；
3. 使用gamepad控制器，在图像上放置一个“绿点”，这个绿点代表当机器人在当前的位置上是想向左还是向右行驶；
4. 存储机器人拍摄的图像，同时存储这个绿色点的X、Y的坐标值作为图片的名称；
5. 在训练的时候，我们将机器人保存的图像作为训练样本，每张图片都有对应X, Y值作为标签，然后把样本和标签输入到神经网络种就可以训练出模型了；
6. 在现场演示中，我们将通过机器人对拍摄到的图片通过模型进行预测，再根据预测出来的X、Y值来计算一个相应转向值。 


# 数据收集

那么，我们具体应该怎样来收集图片呢？
执行下面这段代码来打开视频进行参考，查看如何对图像进行标记。

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event,bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from traitlets.config.configurable import Configurable
from IPython.display import display
from uuid import uuid1
import os
import json
import glob
import numpy as np
import cv2

## 2.连接机器人

In [ ]:
rbt = Robot()
rbt.connect()

## 3.创建文件夹

In [ ]:
# 创建文件夹
DATASET_DIR = 'dataset_xy'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

## 4.在图像上创建绿色标记
1. 创建图像显示窗口；
2. 获取图像数据；
3. 在图像上创建一个绿点在标记时作为参考；
4. 显示图形化界面。

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
target_widget = widgets.Image(format='jpeg', width=300, height=300)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.0001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.0001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 300 / 2 + 150)
    y = int(y * 300 / 2 + 150)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    #img:图像，圆心坐标，圆半径，颜色，线宽度(-1：表示对封闭图像进行内部填满)
    image = cv2.circle(image, (150, 300), 8, (0, 0,255), 3)
    # img:图像，起点坐标，终点坐标，颜色，线的宽度
    image = cv2.line(image, (x,y), (150, 300), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

def on_new_image(evt):
    image_widget.value= bgr8_to_jpeg(evt.dict['data'])
    target_widget.value = display_xy(evt.dict['data'])
    
rbt.camera.start()
rbt.base.set_ptz(-20)
rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

## 5.创建gamepad控制器

我们可以通过游戏手柄来控制机器人，首先将游戏手柄的无线usb插到电脑上，然后打开游戏手柄的开关。http://html5gamepad.com 打开这个网址然后按下你正在使用的游戏手柄上的按钮，可以看到网页上也会有相应的反应。
在网站上可以看到“index”下面有个数字，记住这个数字，然后我们通过“widgets.Controller()”连接到手柄然后再显示出来。

In [ ]:
controller = widgets.Controller(index=0)
display(controller)

## 6.连接Gamepad控制器来标记图像

虽然我们现在已经创建了gamepad控制器，但我们还需要将它连接到上下左右滑块，来控制绿点的位置，我们将使用dlink函数将手柄的摇杆其连接到左右垂直滑块。
然后我们再用gamepad控制器的另外一个摇杆连接到机器人，这样我们就可以通过手柄来控制小车，并且收集数据了。

In [ ]:
class Move(Configurable):
    x_speed = traitlets.Float(default_value=0.0)
    a_speed = traitlets.Float(default_value=0.0)
    @traitlets.observe('x_speed')
    def x_speed_value(self, change):
        self.x_speed=change['new']
        rbt.base.move(x_speed=self.x_speed)

    @traitlets.observe('a_speed')
    def a_speed_value(self, change):
        self.a_speed=change['new']
        rbt.base.move(a_speed=self.a_speed)
move=Move()

# 将手柄上的按钮axes[3]来控制小车的前进后退，按钮axes[2]来控制小车的左右。
move_link = traitlets.dlink((controller.axes[3], 'value'), (move, 'x_speed'), transform=lambda x: -x)
turn_link = traitlets.dlink((controller.axes[2], 'value'), (move, 'a_speed'), transform=lambda x: -x)

# 将手柄上的按钮axes[0]连接到左右滑块，按钮axes[1]连接到上下滑块
widgets.jsdlink((controller.axes[0], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[1], 'value'), (y_slider, 'value'))

## 7.开始收集数据

In [ ]:
# 在窗口实时显示图片的数量
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

# 给图片的添加一个随机不重复的文件名(注意X, Y的位置都是固定的，因为当我们训练时，需要先加载图像并解析文件名中的x、y值)。
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

# 创建保存按钮并连接到游戏手柄上，通过点击buttons[1]就可以保存图片了
def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[1].observe(save_snapshot, names='value')

# 显示可视化窗口
display(x_slider, y_slider)
display(widgets.VBox([target_widget,count_widget]))

当你收集了最够多的数据后，运行下面代码与机器人断开连接。

In [ ]:
# rbt.disconnect()